In [ ]:
# Load necessary libraries
library(dplyr)
library(Matrix)
library(data.table)
library(Seurat)
library(ggplot2)
library(RColorBrewer)
library(cowplot)
library(tidyverse)
library(tidyr)
library(viridis)
library(harmony)
library(MASS) #
library(scales)
library(AnnotationHub)	
library(org.Hs.eg.db)  
library(clusterProfiler)
library(biomaRt)
library(ggthemes)
library(msigdbr)
library(conflicted)
library(stringr)


In [ ]:
###

# Define the list of cell types for analysis
celltypelist = c('EX_CA1', 'EX_CA2', 'EX_CA3', 'EX_CA4')  # You can add more cell types here, e.g.,

# Loop over each cell type for enrichment analysis
for (celltype in celltypelist) {
  # Read the differentially expressed genes (DEGs) file
  scRNA1.markers = read.csv(paste0('./DEGs/', celltype, '_ADvsCON_DEGS.csv'))
  
  # Extract upregulated genes with log2FC > 0.25 and adjusted p-value < 0.05
  scRNA1.marker = scRNA1.markers %>%
    filter(avg_log2FC > 0.25, UPvsDown == "up", p_val_adj < 0.05)
  
  ##### Enrichment analysis for upregulated genes
  if (nrow(scRNA1.marker) > 0) {
    # Prepare gene list for enrichment analysis
    scRNA1.marker$gene = scRNA1.marker$X
    mygene <- AnnotationDbi::select(org.Hs.eg.db, columns = c("SYMBOL", "ENTREZID"), keytype = "SYMBOL", keys = scRNA1.marker$gene)
    mygene <- mygene$ENTREZID
    
    # Perform GO enrichment analysis
    ego <- enrichGO(gene = mygene, OrgDb = org.Hs.eg.db, ont = "ALL", pAdjustMethod = "BH", pvalueCutoff = 0.05, qvalueCutoff = 0.2, readable = TRUE)
    GO.list = ego@result
    GO.list = GO.list[GO.list$pvalue < 0.05,]
    
  }
  
  # Extract downregulated genes with log2FC < -0.25 and adjusted p-value < 0.05
  scRNA1.marker = scRNA1.markers %>%
    filter(avg_log2FC < -0.25, UPvsDown == "down", p_val_adj < 0.05)
  
  ##### Enrichment analysis for downregulated genes
  if (nrow(scRNA1.marker) > 0) {
    # Prepare gene list for enrichment analysis
    scRNA1.marker$gene = scRNA1.marker$X
    mygene <- AnnotationDbi::select(org.Hs.eg.db, columns = c("SYMBOL", "ENTREZID"), keytype = "SYMBOL", keys = scRNA1.marker$gene)
    mygene <- mygene$ENTREZID
    
    # Perform GO enrichment analysis
    ego <- enrichGO(gene = mygene, OrgDb = org.Hs.eg.db, ont = "ALL", pAdjustMethod = "BH", pvalueCutoff = 0.05, qvalueCutoff = 0.2, readable = TRUE)
    GO.list = ego@result
    GO.list = GO.list[GO.list$pvalue < 0.05,]
    
  }
}
